In [1]:
!nvidia-smi

Mon Feb 13 15:03:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.67       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:65:00.0 Off |                  N/A |
| 73%   55C    P8    54W / 370W |  23644MiB / 24265MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:B3:00.0 Off |                  N/A |
| 65%   

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
import pickle
import pandas as pd
import numpy as np
import warnings
from models import *
from evaluation import *
from load_data import *
from resnet3d import *

print(tf.__version__)
warnings.filterwarnings("ignore")

gpus = tf.config.list_physical_devices(device_type='GPU')
tf.config.set_visible_devices(devices=gpus[0], device_type='GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

2.5.0


In [3]:
seed = 2021
os.environ['PYTHONHASHSEED']=str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

# Evaluate the predictions

## 2D CXR

In [4]:
def get_data(aug_method='_rotation90', dataset='mimic', data_split='test', task='disease', return_demo=False):
    
    np.random.seed(2021)
            
    X = []
    y = []
    demo = []
    
    filename = 'data/{dataset}_{data_split}{aug_method}.tfrecords'.format(dataset=dataset, data_split=data_split, aug_method=aug_method)

    raw_dataset = tf.data.TFRecordDataset(filename)
    for raw_record in raw_dataset:
        label = []
        
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        
        race = example.features.feature['race'].int64_list.value[0]
        age = example.features.feature['age'].int64_list.value[0]
        if (age > 0 and dataset == 'mimic'):
            age -= 1
        gender = example.features.feature['gender'].int64_list.value[0]
        
        temp = [race, gender, age]
#         {"race":race, "gender":gender, "age":age}
        demo.append(temp)
                        
        if (task=='disease'):
            
            label.append(1 if example.features.feature['Atelectasis'].float_list.value[0] == 1 else 0)
            label.append(1 if example.features.feature['Cardiomegaly'].float_list.value[0] == 1 else 0)
            label.append(1 if example.features.feature['Consolidation'].float_list.value[0] == 1 else 0)
            label.append(1 if example.features.feature['Edema'].float_list.value[0] == 1 else 0)
            label.append(1 if example.features.feature['Enlarged Cardiomediastinum'].float_list.value[0] == 1 else 0)
            label.append(1 if example.features.feature['Fracture'].float_list.value[0] == 1 else 0)
            label.append(1 if example.features.feature['Lung Lesion'].float_list.value[0] == 1 else 0)
            label.append(1 if example.features.feature['Lung Opacity'].float_list.value[0] == 1 else 0)
            label.append(1 if example.features.feature['No Finding'].float_list.value[0] == 1 else 0)
            label.append(1 if example.features.feature['Pleural Effusion'].float_list.value[0] == 1 else 0)
            label.append(1 if example.features.feature['Pleural Other'].float_list.value[0] == 1 else 0)
            label.append(1 if example.features.feature['Pneumonia'].float_list.value[0] == 1 else 0)
            label.append(1 if example.features.feature['Pneumothorax'].float_list.value[0] == 1 else 0)
            label.append(1 if example.features.feature['Support Devices'].float_list.value[0] == 1 else 0)
            
        elif (task == 'race'):
            
            if (race == 0):
                label = [1, 0, 0]
            elif (race == 1):
                label = [0, 1, 0]
            else:
                label = [0, 0, 1]
                
        elif (task == 'age'):
            
            if (age == 0):
                label = [1, 0, 0, 0]
            elif (age == 1):
                label = [0, 1, 0, 0]
            elif (age == 2):
                label = [0, 0, 1, 0]
            else:
                label = [0, 0, 0, 1]
                
        elif (task == 'gender'):
            
            if (gender == 0):
                label = [1, 0]
            else:
                label = [0, 1]
                
        else:
            raise NameError('Wrong task')

#         nparr = np.fromstring(example.features.feature['jpg_bytes'].bytes_list.value[0], np.uint8)
#         img_np = cv.imdecode(nparr, cv.IMREAD_GRAYSCALE)  
        
        X.append(0)
                        
        y.append(label)
                
    if (return_demo):
        return np.array(X), np.array(y), np.array(demo)
    else:
        return np.array(X), np.array(y)
    
aug_method = ''
dataset = 'mimic'
task = 'disease'

X_test, y_test, demo = get_data(aug_method=aug_method, dataset=dataset, data_split='test', task=task, return_demo=True)

In [6]:
def compute_result(y_preds, y_test, best_thresh, demo, result_name, group):
    group_type = {'race': [0, 1, 4], 'gender': [0, 1], 'age': [0, 1, 2, 3]}
    dfs = []
    for j in group_type[group]:

        idx = demo[:, list(group_type.keys()).index(group)] == j

        dfs_disease = []
        for k in range(14):

            dfs_disease.append(test(y_preds[idx, k], y_test[idx, k], best_thresh[k]))

        dfs.append(dfs_disease)

    with open("results/{i}_{group}_results".format(i=result_name, group=group), "wb") as fp:
        pickle.dump(dfs, fp)

In [7]:
model_name = 'densenet_mimic_ERM_proposed_no_weight'
result_name = '{i}_on_original'.format(i=model_name)
prediction_name = 'predictions/model_{i}_on_original'.format(i=model_name)
best_thresh = np.loadtxt('thresh/model_{i}_thresh.txt'.format(i=model_name))

with open(prediction_name, "rb") as fp:
    y_preds = pickle.load(fp)
fp.close()
    
compute_result(y_preds, y_test, best_thresh, demo, result_name, 'race')
compute_result(y_preds, y_test, best_thresh, demo, result_name, 'gender')
compute_result(y_preds, y_test, best_thresh, demo, result_name, 'age')

In [8]:
model_name = 'densenet_mimic_ERM_proposed_no_weight'
result_name = '{i}_on_aug'.format(i=model_name)
prediction_name = 'predictions/model_{i}_on_aug'.format(i=model_name)
best_thresh = np.loadtxt('thresh/model_{i}_thresh.txt'.format(i=model_name))

with open(prediction_name, "rb") as fp:
    y_preds = pickle.load(fp)
fp.close()
    
compute_result(y_preds, y_test, best_thresh, demo, result_name, 'race')
compute_result(y_preds, y_test, best_thresh, demo, result_name, 'gender')
compute_result(y_preds, y_test, best_thresh, demo, result_name, 'age')

In [9]:
model_name = 'densenet_mimic_ERM_no_weight'
result_name = '{i}_on_original'.format(i=model_name)
prediction_name = 'predictions/model_{i}_on_original'.format(i=model_name)
best_thresh = np.loadtxt('thresh/model_{i}_thresh.txt'.format(i=model_name))

with open(prediction_name, "rb") as fp:
    y_preds = pickle.load(fp)
fp.close()
    
compute_result(y_preds, y_test, best_thresh, demo, result_name, 'race')
compute_result(y_preds, y_test, best_thresh, demo, result_name, 'gender')
compute_result(y_preds, y_test, best_thresh, demo, result_name, 'age')

In [10]:
model_name = 'densenet_mimic_ERM_no_weight'
result_name = '{i}_on_aug'.format(i=model_name)
prediction_name = 'predictions/model_{i}_on_aug'.format(i=model_name)
best_thresh = np.loadtxt('thresh/model_{i}_thresh.txt'.format(i=model_name))

with open(prediction_name, "rb") as fp:
    y_preds = pickle.load(fp)
fp.close()
    
compute_result(y_preds, y_test, best_thresh, demo, result_name, 'race')
compute_result(y_preds, y_test, best_thresh, demo, result_name, 'gender')
compute_result(y_preds, y_test, best_thresh, demo, result_name, 'age')

## 3D MRI

In [19]:
df = pd.read_csv('data_new.csv')
data_path = '../../../mnt/usb/kuopc/ADNI_B1/MPR__GradWarp__B1_Correction_crop/'

df = df.loc[df['Group'] != 'MCI']
df = df.loc[df['Split'] == 'test']

df['Group'] = df['Group'].replace(['CN', 'AD'], [0, 1])
df['Sex'] = df['Sex'].replace(['F', 'M'], [0, 1])
df['Age'] = np.where(df['Age'] <= 75, 0, 1)
df['Race'] = np.where(df['Race'] < 1, 0, 1)

In [21]:
model_name = '3D_CNN_AD_CN_age'
prediction_name = 'predictions/{i}_on_original'.format(i=model_name)
with open(prediction_name, "rb") as fp:
    y_preds = pickle.load(fp)
demo_test(y_preds, df['Age'].values)


model_name = '3D_CNN_AD_CN_age_proposed'
prediction_name = 'predictions/{i}_on_aug'.format(i=model_name)
with open(prediction_name, "rb") as fp:
    y_preds = pickle.load(fp)
demo_test(y_preds, df['Age'].values)

Sample auc mean: 0.687
Samole auc std: 0.033
Sample auc CI: 0.074
Confidence interval for the score: [0.612 - 0.761]
Sample auc mean: 0.691
Samole auc std: 0.033
Sample auc CI: 0.075
Confidence interval for the score: [0.616 - 0.766]


In [22]:
model_name = '3D_CNN_AD_CN_gender'
prediction_name = 'predictions/{i}_on_original'.format(i=model_name)
with open(prediction_name, "rb") as fp:
    y_preds = pickle.load(fp)
demo_test(y_preds, df['Sex'].values)


model_name = '3D_CNN_AD_CN_gender_proposed'
prediction_name = 'predictions/{i}_on_aug'.format(i=model_name)
with open(prediction_name, "rb") as fp:
    y_preds = pickle.load(fp)
demo_test(y_preds, df['Sex'].values)

Sample auc mean: 0.856
Samole auc std: 0.025
Sample auc CI: 0.056
Confidence interval for the score: [0.800 - 0.912]
Sample auc mean: 0.857
Samole auc std: 0.024
Sample auc CI: 0.053
Confidence interval for the score: [0.803 - 0.910]


In [11]:
def compute_result(y_preds, y_test, best_thresh, demo, result_name, group):
    group_type = {'race': [0, 1], 'gender': [0, 1], 'age': [0, 1]}
    dfs = []

    for j in group_type[group]:

        idx = demo == j
        
        dfs.append(test(y_preds[idx], y_test[idx], best_thresh))

    with open("results/{i}_{group}_results".format(i=result_name, group=group), "wb") as fp:
        pickle.dump(dfs, fp)

In [15]:
model_name = '3D_CNN_AD_CN_balanced'
result_name = '{i}_on_original'.format(i=model_name)
prediction_name = 'predictions/{i}_on_original'.format(i=model_name)
best_thresh = np.loadtxt('thresh/{i}_thresh.txt'.format(i=model_name))

with open(prediction_name, "rb") as fp:
    y_preds = pickle.load(fp)
    
compute_result(y_preds, df['Group'].values, best_thresh, df['Sex'].values, result_name, 'gender')
compute_result(y_preds, df['Group'].values, best_thresh, df['Age'].values, result_name, 'age')

In [16]:
model_name = '3D_CNN_AD_CN_balanced_proposed'
result_name = '{i}_on_original'.format(i=model_name)
prediction_name = 'predictions/{i}_on_original'.format(i=model_name)
best_thresh = np.loadtxt('thresh/{i}_thresh.txt'.format(i=model_name))

with open(prediction_name, "rb") as fp:
    y_preds = pickle.load(fp)
    
compute_result(y_preds, df['Group'].values, best_thresh, df['Sex'].values, result_name, 'gender')
compute_result(y_preds, df['Group'].values, best_thresh, df['Age'].values, result_name, 'age')

In [17]:
model_name = '3D_CNN_AD_CN_balanced'
result_name = '{i}_on_aug'.format(i=model_name)
prediction_name = 'predictions/{i}_on_aug'.format(i=model_name)
best_thresh = np.loadtxt('thresh/{i}_thresh.txt'.format(i=model_name))

with open(prediction_name, "rb") as fp:
    y_preds = pickle.load(fp)

compute_result(y_preds, df['Group'].values, best_thresh, df['Sex'].values, result_name, 'gender')
compute_result(y_preds, df['Group'].values, best_thresh, df['Age'].values, result_name, 'age')

In [18]:
model_name = '3D_CNN_AD_CN_balanced_proposed'
result_name = '{i}_on_aug'.format(i=model_name)
prediction_name = 'predictions/{i}_on_aug'.format(i=model_name)
best_thresh = np.loadtxt('thresh/{i}_thresh.txt'.format(i=model_name))

with open(prediction_name, "rb") as fp:
    y_preds = pickle.load(fp)

compute_result(y_preds, df['Group'].values, best_thresh, df['Sex'].values, result_name, 'gender')
compute_result(y_preds, df['Group'].values, best_thresh, df['Age'].values, result_name, 'age')